In [2]:
import folium
from folium import plugins
import greppo
import numpy as np
import pandas as pd
import geojson
import requests
from bs4 import BeautifulSoup as bfs
import re

In [3]:
# load germany geojson data
with open('/home/jan/Uni/DS-Project/data/Maps/Bundeskarte.geojson') as file:
    bund_map = geojson.load(file)
with open('/home/jan/Uni/DS-Project/data/Maps/Laenderkarte.geojson') as file:
    laender_map = geojson.load(file)
with open('/home/jan/Uni/DS-Project/data/Maps/Kreiskarte.geojson') as file:
    kreis_map = geojson.load(file)
# load cities
with open('/home/jan/Uni/DS-Project/data/Maps/Staedte.geojson') as file:
    staedte_map = geojson.load(file)

In [4]:
Germany = folium.Map(location=[51.2, 10.5],
                 min_lon=5,
                 max_lon=15.7,
                 min_lat=55.5,
                 max_lat=46.5,
                 min_zoom=6,
                 zoom_start=6,
                 max_bounds=True,
                 control_scale=True,
                 tiles = "OpenStreetMap")
# implement extras
minimap = plugins.MiniMap(toggle_display = True)
Germany.add_child(minimap)
plugins.Fullscreen(position = 'topright').add_to(Germany)
plugins.ScrollZoomToggler().add_to(Germany)

In [5]:
folium.GeoJson(kreis_map).add_to(Germany)
#folium.GeoJson(staedte_map).add_to(map)

Germany.save('Germany_borders.html')

In [ ]:
Germany

# test area

In [ ]:

drop_Bund = folium.FeatureGroup(name="Bund")

drop_Bund.add_child(folium.GeoJson(data = bund_map,
                                   style_function = lambda x: {
                                       'fillColor': get_at_once(x['properties']['NAME'], 'Bund'),
                                       'color' = 'black',
                                       'fillOpacity' = .75
                                   }))

drop_Staedte = folium.FeatureGroup(name="Staedte", show=False)

drop_Staedte.add_child(folium.GeoJson(data = staedte_map,
                                   style_function = lambda x: {
                                       'fillColor': get_at_once(x['properties']['NAME'], 'Staedte'),
                                       'color' = 'pink',
                                       'fillOpacity' = .5
                                   }))

map.add_child(drop_Bund)
map.add_child(drop_Staedte)
map.add_child(folium.LayerControl())
map.save('germany_borders.html')

In [5]:
test = kreis_map

test['features'][0]['properties']#['NAME_3']



{'ID_0': 86,
 'ISO': 'DEU',
 'NAME_0': 'Germany',
 'ID_1': 9,
 'NAME_1': 'Niedersachsen',
 'ID_2': 23,
 'NAME_2': 'Weser-Ems',
 'ID_3': 244,
 'NAME_3': 'Oldenburg',
 'NL_NAME_3': None,
 'VARNAME_3': None,
 'TYPE_3': 'Landkreise',
 'ENGTYPE_3': 'Rural district'}

In [19]:
staedte_map['features'][2]['properties']['NAME']

'HONNINGSVAG'

In [7]:
city_list = pd.read_csv('/home/jan/Uni/DS-Project/data/Maps/worldcities.csv')
city_list = city_list[['city', 'lat', 'lng', 'country', 'population']]

In [8]:
german_cities = city_list[(city_list.country == 'Germany') & (city_list.population > 100000)]

In [9]:
german_staedte_map = []
for city in german_cities.city:
    print(city)

Berlin
Hamburg
Munich
Cologne
Frankfurt
Düsseldorf
Stuttgart
Leipzig
Dortmund
Essen
Bremen
Dresden
Hannover
Nuremberg
Duisburg
Bochum
Wuppertal
Bielefeld
Bonn
Münster
Mannheim
Karlsruhe
Augsburg
Wiesbaden
Gelsenkirchen
Mönchengladbach
Aachen
Braunschweig
Kiel
Chemnitz
Halle
Magdeburg
Freiburg im Breisgau
Krefeld
Mainz
Lübeck
Erfurt
Oberhausen
Rostock
Kassel
Hagen
Potsdam
Saarbrücken
Hamm
Ludwigshafen
Mülheim
Oldenburg
Osnabrück
Leverkusen
Solingen
Darmstadt
Heidelberg
Herne
Neuss
Regensburg
Paderborn
Ingolstadt
Fürth
Würzburg
Heilbronn
Ulm
Pforzheim
Wolfsburg
Göttingen
Bottrop
Reutlingen
Koblenz
Bremerhaven
Recklinghausen
Erlangen
Bergisch Gladbach
Remscheid
Jena
Trier
Moers
Salzgitter
Siegen
Hildesheim
Gütersloh
Kaiserslautern


In [10]:
wikiurl = 'https://de.wikipedia.org/wiki/Liste_der_Gro%C3%9Fst%C3%A4dte_in_Deutschland'
table_class = 'wikitable sortable jquery-tablesorter'
response = requests.get(wikiurl)
print(response.status_code)

200


In [11]:
soup = bfs(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [12]:
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])[['Name']]

german_cities = df.Name.Name

In [17]:
german_cities_map = []
for city in german_cities:
    german_city = re.sub(r'[^a-zA-ZäöüßÄÖÜ]+', '', city)
    print(german_city)
    german_cities_map.append(german_city)

Berlin
Hamburg
München
Köln
FrankfurtamMain
Stuttgart
Düsseldorf
Leipzig
Dortmund
Essen
Bremen
Dresden
Hannover
Nürnberg
Duisburg
Bochum
Wuppertal
Bielefeld
Bonn
Münster
Mannheim
Karlsruhe
Augsburg
Wiesbaden
Mönchengladbach
Gelsenkirchen
Aachen
Braunschweig
Kiel
Chemnitz
HalleSaale
Magdeburg
FreiburgimBreisgau
Krefeld
Mainz
Lübeck
Erfurt
Oberhausen
Rostock
Kassel
Hagen
Potsdam
Saarbrücken
Hamm
LudwigshafenamRhein
MülheimanderRuhr
OldenburgOldb
Osnabrück
Leverkusen
Darmstadt
Heidelberg
Solingen
Herne
Regensburg
Neuss
Paderborn
Ingolstadt
OffenbachamMain
Fürth
Würzburg
Ulm
Heilbronn
Pforzheim
Wolfsburg
Bottrop
Göttingen
Reutlingen
Koblenz
Erlangen
Bremerhaven
Remscheid
BergischGladbach
Recklinghausen
Trier
Jena
Moers
Salzgitter
Siegen
Gütersloh
Hildesheim
Name
Name
